In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.signal import savgol_filter

In [3]:
data_traits = pd.read_csv('2022_sorghum_traits.csv', dtype = { 'Leaf_Num' : 'category', 'SLA' : 'category' , 'THCK' : 'category', 'LWC' : 'category'})

In [4]:
data_reflectance = pd.read_csv('Sorghum_2022_reflectance.csv', dtype={'Genotype' : 'category', 'Rep': 'category' , 'Leaf_Num' : 'category', 'Leaf_pos' : 'category'})

In [5]:
data_traits.head()

,Genotype ID,Accession,Genotype,Rep,Leaf_num,CHLp1,CHLp2,CHLp3,FW,LA,DW,SLA,THCK,LWC
0,GREENLEAF,PI 659692,1,1,1,367.4,472.9,399.7,4.45,281.59,1.11,253.684684684685,63.2786516853933,75.0561797752809
1,GREENLEAF,PI 659692,1,1,2,399.7,739.3,357.8,3.45,225.36,0.99,227.636363636364,65.3217391304348,71.304347826087
2,GREENLEAF,PI 659692,1,1,3,439.2,362.3,347.7,1.65,125.04,0.48,260.5,75.7818181818182,70.9090909090909
3,GREENLEAF,PI 659692,1,2,1,447.1,141.9,160.9,4.45,249.96,1.06,235.811320754717,56.1707865168539,76.1797752808989
4,GREENLEAF,PI 659692,1,2,2,325.2,481.0,322.5,1.90,144.58,0.55,262.872727272727,76.0947368421053,71.0526315789474


In [6]:
data_reflectance.head()

,ASD_Label,Genotype,Rep,Leaf_num,Leaf_pos,350,351,352,353,354,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,1-1-1-3,1,1,1,3,0.067603,0.097000,0.104572,0.0956,0.089000,...,0.072200,0.071300,0.070700,0.069800,0.069483,0.070500,0.070100,0.068500,0.068300,0.068400
1,1-2-2-1,1,2,2,1,0.086900,0.111040,0.103771,0.0802,0.091000,...,0.059800,0.060100,0.059700,0.058400,0.057900,0.057200,0.057000,0.058400,0.058300,0.057200
2,1-2-3-3,1,2,3,3,0.090700,0.074000,0.066400,0.0781,0.097600,...,0.068200,0.065967,0.065500,0.065100,0.064900,0.066700,0.066800,0.065400,0.064600,0.062800
3,1-2-1-2,1,2,1,2,0.097900,0.112459,0.103933,0.0905,0.118572,...,0.150094,0.149474,0.149433,0.150717,0.150821,0.150798,0.150168,0.148169,0.147093,0.145641
4,1-2-3-2,1,2,3,2,0.100644,0.106306,0.097900,0.0850,0.098500,...,0.051200,0.050941,0.050500,0.049700,0.049500,0.049623,0.049200,0.048500,0.048277,0.047548


In [7]:
data_traits = data_traits.iloc[: , 0:14]
data_traits.head()

,Genotype ID,Accession,Genotype,Rep,Leaf_num,CHLp1,CHLp2,CHLp3,FW,LA,DW,SLA,THCK,LWC
0,GREENLEAF,PI 659692,1,1,1,367.4,472.9,399.7,4.45,281.59,1.11,253.684684684685,63.2786516853933,75.0561797752809
1,GREENLEAF,PI 659692,1,1,2,399.7,739.3,357.8,3.45,225.36,0.99,227.636363636364,65.3217391304348,71.304347826087
2,GREENLEAF,PI 659692,1,1,3,439.2,362.3,347.7,1.65,125.04,0.48,260.5,75.7818181818182,70.9090909090909
3,GREENLEAF,PI 659692,1,2,1,447.1,141.9,160.9,4.45,249.96,1.06,235.811320754717,56.1707865168539,76.1797752808989
4,GREENLEAF,PI 659692,1,2,2,325.2,481.0,322.5,1.90,144.58,0.55,262.872727272727,76.0947368421053,71.0526315789474


In [8]:
data_traits = data_traits.loc[data_traits['DW'].notnull() , :]

In [9]:
data_traits.shape

(353, 14)

In [10]:
data_traits['Genotype'].dtype

dtype('int64')

In [11]:
grouped = data_traits.groupby(['Genotype', 'Rep']).mean()

In [12]:
grouped.head()

Leaf_num       CHLp1       CHLp2       CHLp3         FW  \
Genotype Rep                                                            
1        1         2.0  402.100000  524.833333  368.400000   3.183333   
         2         2.0  368.433333  323.066667  254.166667   2.683333   
2        1         1.5  778.500000  750.350000  631.400000  16.000000   
         2         2.0  581.900000  429.000000  216.466667  16.483333   
4        1         2.0  510.433333  557.733333  555.100000  13.833333   

                      LA        DW  
Genotype Rep                        
1        1    210.663333  0.860000  
         2    175.103333  0.700000  
2        1    617.350000  4.215000  
         2    630.256667  4.650000  
4        1    596.180000  4.443333

In [83]:
grouped.shape

(120, 7)

In [47]:
data = pd.merge(data_traits, data_reflectance, how='outer')

In [48]:
data = data.loc[data['CHLp1'].notnull() , :]

In [49]:
data.to_csv(r'/home/schnable/Downloads/sorghum_2022_whole.csv', index = None)